# KoBERT finetuning

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 82.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 35.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=3c1a41d31970e8f9e4e022412d8a94abaf667b2398c5ed8cf7e83feb33966bd9
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 29.3 MB/s 
     |████████████████████████████████| 769 kB 34.1 MB/s 
     |████████████████████████████████| 3.0 MB 63.6 MB/s 
     |████████████████████████████████| 895 kB 59.5 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-l8l09sm4
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-l8l09sm4
     |████████████████████████████████| 132 kB 42.1 MB/s 
     |████████████████████████████████| 4.5 MB 67.0 MB/s 
     |████████████████████████████████| 4.0 MB 69.0 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
     |████████████████████████████████| 77 kB 8.4 MB/s 
     |████████████████████████████████| 6.6 MB 59.3 MB/s 
     |████████████████████████████████| 8.7 MB 43.4 MB/s 
     |████████████████████████████████| 79 kB 10.6 MB/s 
     |████████████████████████████████| 127 kB 96.1 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=f1e667afe57bc1216abfc47842c4b22ce2206b9dab40b26713ec71ce12c3596c
  Stored in directory: /tmp/pip-ephem-wheel-cache-yiijpkka/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
## CPU
device = torch.device("cpu")

# GPU
device = torch.device("cuda:0")

In [ ]:
# GPU 확인하기
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
feeling_df = pd.read_excel('.\kimmaa_nlp\한국어 감정 정보가 포함된 단발성 대화 데이터셋\한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
print(len(feeling_df))
feeling_df.sample(n=10)

38594


,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
19882,도깨비 살려주세요 ㅠㅠ,슬픔,NaN,NaN,NaN,NaN,NaN
13021,후안이 무치한 뇬,분노,NaN,NaN,NaN,NaN,NaN
31125,jtbc가 최고네요. 수고 많으십니다.,행복,NaN,NaN,NaN,NaN,NaN
9858,화웨이도 엣지넣고싶어서 그러는거군...,놀람,NaN,NaN,NaN,NaN,NaN
19466,불펜 투수의 몸값이 하늘 높이 오르는군...,슬픔,NaN,NaN,NaN,NaN,NaN
20564,너무빡치고 힘든데 어떡하나요,슬픔,NaN,NaN,NaN,NaN,NaN
13929,순복음 교회 조용기가 단연 쓰레기 중 쓰레기지.,분노,NaN,NaN,NaN,NaN,NaN
25235,케이블이20넘으면 지상파였으면 더됬을까?ㅋㅋ,중립,NaN,NaN,NaN,NaN,NaN
22222,말만 하면 분위기가 그냥 숙연해짐 ㅠㅠ,슬픔,NaN,NaN,NaN,NaN,NaN
23242,김국진성격에이정도오픈하는걸보니결혼할생각인듯,중립,NaN,NaN,NaN,NaN,NaN


In [ ]:
feeling_df.loc[(feeling_df['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
feeling_df.loc[(feeling_df['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
feeling_df.loc[(feeling_df['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
feeling_df.loc[(feeling_df['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
feeling_df.loc[(feeling_df['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
feeling_df.loc[(feeling_df['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
feeling_df.loc[(feeling_df['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [ ]:
df_list = []
for q, label in zip(feeling_df['Sentence'], feeling_df['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    df_list.append(data)


In [ ]:
feeling_df.sample(n=10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
37686,어딜가나 에이전트들이 문제구만‥‥ 야야투레부터 시작해서,6,NaN,NaN,NaN,NaN,NaN
31327,천진난만해보인다 ㅎㅎㅎㅎㅎ,5,NaN,NaN,NaN,NaN,NaN
2815,그래서 8년우정이깨지고있는데 불안해요,0,NaN,NaN,NaN,NaN,NaN
14692,증거 좀 달라 하면 주지도 못할 것들이앞담은 드럽게 까요,2,NaN,NaN,NaN,NaN,NaN
37205,지능이 모자라서 섹스 먼저 하지,6,NaN,NaN,NaN,NaN,NaN
33508,그 더러운 손에 감히 태극기를 들지말라.,6,NaN,NaN,NaN,NaN,NaN
37788,저건 뭐 내 주먹으로도 부서지겠다,6,NaN,NaN,NaN,NaN,NaN
11772,저 여자는 왜 아직도 근무하지?,2,NaN,NaN,NaN,NaN,NaN
20233,어른이 세상에서 가장 힘든 역할인걸 깨달은 요즘..,3,NaN,NaN,NaN,NaN,NaN
28681,고객들에게 빨리 배송하기위해 고생하는 택배기사님들 정말 고마운분들이다,5,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(df_list[0])
print(df_list[6000])
print(df_list[12000])
print(df_list[18000])
print(df_list[24000])
print(df_list[30000])
print(df_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그거들은 뒤부터 미치겠어요...', '3']
['최악의 상황중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
df_train, df_test = train_test_split(df_list, test_size=0.25, random_state=0)
print(len(df_train))
print(len(df_test))

28945
9649


In [ ]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# tokenizer = get_tokenizer("beomi/kcbert-base")
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(df_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(df_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
print(data_train[0])
print(data_test[0])

(array([   2, 1189,  517, 6188, 7245, 7063,  517,  463, 3486, 7836, 5966,
       1698,  517, 6188, 7245, 7063,  517,  463, 1281, 7870, 1801, 6885,
       7088, 5966, 1698, 5837, 5837,  517, 6188, 7245, 6398, 6037, 7063,
        517,  463,  517,  463,  517,  364,  517,  364,    3,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32), array(42, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32), 5)
(array([   2,  520, 3220, 7096, 6882, 3036, 7408,  520,  517, 7102, 3095,
       5330, 3396, 5561, 7782, 5837, 5512, 6999,  517,   54,    3,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
    

# KoBERT 학습모델 만들기


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_dataloader

# KoBERT 모델 학습시키기

In [ ]:
PATH = '/content/drive/Othercomputers/내 노트북/PY4E/DS/snsscrape/weights/' # 폴더가 없을 경우 만들어줘야함!!!!

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0179479122161865 train acc 0.234375
epoch 1 batch id 201 loss 1.4301248788833618 train acc 0.22349191542288557
epoch 1 batch id 401 loss 1.2763290405273438 train acc 0.33451527431421446
epoch 1 train acc 0.3528782787949617


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 1 test acc 0.4984921948912015


...

epoch 9 test acc 0.5329014055953507


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.05417780578136444 train acc 0.96875
epoch 10 batch id 201 loss 0.14483416080474854 train acc 0.9594993781094527
epoch 10 batch id 401 loss 0.14690512418746948 train acc 0.9630221321695761
epoch 10 train acc 0.9625413907284768


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 10 test acc 0.5339044972293553


train acc 0.9785113134657837

test acc 0.529081210298689


//
epoch 5 train acc 0.7677452603557979
100%
151/151 [01:14<00:00, 2.19it/s]
epoch 5 test acc 0.5433293350452764
//

epoch 10 test acc 0.5363245877821328 
epoch 10 train acc 0.9587817328918322



In [ ]:
torch.save(
    {
    'epoch' : e+1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
    },
    PATH + 'model_{}.tar'.format(e+1) 
    )

## check-point 만들기


In [ ]:
'''# Define the Huggingface Trainer object
#model = model
#args = 상단에서 선언한 traing_args
#train_dataset과 eval_data_set에서는 토치텐서로 변형된 데이터를 넣어준다.
trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_set_dataset,
    eval_dataset    = valid_set_dataset,
    compute_metrics = compute_metrics,
    callbacks = callbacks
)
'''

'# Define the Huggingface Trainer object\n#model = model\n#args = 상단에서 선언한 traing_args\n#train_dataset과 eval_data_set에서는 토치텐서로 변형된 데이터를 넣어준다.\ntrainer = Trainer(\n    model           = model,\n    args            = training_args,\n    train_dataset   = train_set_dataset,\n    eval_dataset    = valid_set_dataset,\n    compute_metrics = compute_metrics,\n    callbacks = callbacks\n)\n'

# KoBERT 모델 예측

In [ ]:
#데이터 불러오기
import pandas as pd
# df_raw = pd.read_csv('/content/drive/Othercomputers/내 노트북/PY4E/DS/snsscrape/1번_twits_전처리완료.csv')
# pred_df = pd.DataFrame(df_raw, columns = ['Datetime', 'Username', 'Like Count', 'Text'])
# pred_df

,Datetime,Username,Like Count,Text
0,2022-03-09,the_b017,2,;- 1번 뽑혔을때 ...
1,2022-03-09,Ruee_3012,0,#상큼발랄하게~#오늘도~#1번 ! #이러기
2,2022-03-09,huaaaaang1,0,#1번 뽑고 멋지게 떡쳐라 !!!
3,2022-03-09,sunday0317,82,“이재명후보 믿고 여기까지 왔습니다!!” 민주당은 박지현씨를 철저하게 지켜주고 지...
4,2022-03-09,semilee98,0,오늘은 세계 여성의 날 입니다. 인류 역사상 여성들은 항상 역사의 그늘에 가려져 있...
...,...,...,...,...
131,2022-02-17,sena77kim,0,#일을하고싶은 #1번_이재명
132,2022-02-17,yj6747,0,_ _ #1번 #울림마트
133,2022-02-15,jobsul,0,여러분 1번입니다. 1번을 찍어야 나라가 삽니다. #윤석열 #1번
134,2022-02-15,subsky888,0,"2월 일(화)""선거운동 첫 날"" 눈이 내리는 선거운동 첫 날! 운동원으로 등록하..."


In [ ]:
# # 2번 
# #데이터 불러오기
# import pandas as pd
# df_raw = pd.read_csv('/content/drive/Othercomputers/내 노트북/PY4E/DS/snsscrape/2번_twits_전처리완료.csv')
# pred_df = pd.DataFrame(df_raw, columns = ['Datetime', 'Username', 'Like Count', 'Text'])
# pred_df

In [ ]:
# 1번남 
#데이터 불러오기
import pandas as pd
# df_raw = pd.read_csv('/content/drive/Othercomputers/내 노트북/PY4E/DS/snsscrape/1번남_twits_전처리완료.csv')
# pred_df = pd.DataFrame(df_raw, columns = ['Datetime', 'Username', 'Like Count', 'Text'])
df_raw = pd.read_csv('/content/drive/Othercomputers/내 노트북/PY4E/DS/snsscrape/1번남_라벨링.csv')
pred_df = pd.DataFrame(df_raw, columns = ['Datetime', 'Username', 'Like Count', 'Text','pred'])
pred_df

,Datetime,Username,Like Count,Text,pred
0,2022-03-09,mingyulover_k,1,2번남이든 1번남이든.. 2번남이라고 생각드면 걍 탈빠하세요 개지랄하지말고,0
1,2022-03-09,kero_commu,1,"그래도 우리 긍정적으로 생각하자고 막판에 1번남 2번남 이슈가 되면서 여초에서도 ""...",0
2,2022-03-09,Manyong_123,1,1번남들도 악착같이 살아보자 2번남들은 니들업보야 새끼들아,0
3,2022-03-09,Ops_the_Ops,0,1번남 명상 같은걸 당직자가 올린다거나 하던 분들이 혐오로 당선됐다 하니까 어지러움 ;;,0
4,2022-03-09,play_with_bow,1,1번남이 하루아침에 완성되는 줄 아나 윤석열 됐다며? 여가부 폐지 진짜 하려나? 하...,1
...,...,...,...,...,...
6393,2022-02-26,DDZZ_S2,0,엄마는 1번남,0
6394,2022-02-16,pumo311,2,2번 여캐지? 일러스트 포지션상 주인공이 가운데에 있고 주인공 기준 오른쪽에 서있는...,0
6395,2022-02-13,leemaryellow,113,💝아로하의 선택 💝 1번남 ✨ 2번남 ✨ 3번남 #아스트로 #아로하 # ...,1
6396,2022-02-10,kiimsann,1,오프하면 1번남 할래,0


In [ ]:
# # 2번남 
# #데이터 불러오기
# import pandas as pd
# df_raw = pd.read_csv('/content/drive/Othercomputers/내 노트북/PY4E/DS/snsscrape/2번남_twits_전처리완료.csv')
# pred_df = pd.DataFrame(df_raw, columns = ['Datetime', 'Username', 'Like Count', 'Text'])
# pred_df

-0-0-0-

In [ ]:
sentences = pred_df['Text']
sentences

0               2번남이든 1번남이든.. 2번남이라고 생각드면 걍 탈빠하세요 개지랄하지말고
1       그래도 우리 긍정적으로 생각하자고 막판에 1번남 2번남 이슈가 되면서 여초에서도 "...
2                       1번남들도 악착같이 살아보자 2번남들은 니들업보야 새끼들아 
3       1번남 명상 같은걸 당직자가 올린다거나 하던 분들이 혐오로 당선됐다 하니까 어지러움 ;;
4       1번남이 하루아침에 완성되는 줄 아나 윤석열 됐다며? 여가부 폐지 진짜 하려나? 하...
                              ...                        
6393                                              엄마는 1번남
6394    2번 여캐지? 일러스트 포지션상 주인공이 가운데에 있고 주인공 기준 오른쪽에 서있는...
6395    💝아로하의 선택 💝 1번남 ✨ 2번남 ✨ 3번남     #아스트로 #아로하 #   ...
6396                                          오프하면 1번남 할래
6397                                           아니1번남누구야? 
Name: Text, Length: 6398, dtype: object

In [ ]:
# Remoce unicode and Do Regex again
import re

def clean_tweets(text):
    text = re.sub("RT @[\w]*:","",str(text))
    text = re.sub("@[\w]*","",str(text))
    text = re.sub("https?://[A-Za-z0-9./]*","",str(text))
    text = re.sub("\n","",str(text))
    return str(text)

pred_df['Text'] = pred_df['Text'].apply(lambda x: clean_tweets(x))
pred_df["Text"] = pred_df["Text"].str.replace(pat=r'[^\w]', repl=r'', regex=True)



In [ ]:
sentences = pred_df['Text']
sentences
pred_df

,Datetime,Username,Like Count,Text,pred
0,2022-03-09,mingyulover_k,1,2번남이든1번남이든2번남이라고생각드면걍탈빠하세요개지랄하지말고,0
1,2022-03-09,kero_commu,1,그래도우리긍정적으로생각하자고막판에1번남2번남이슈가되면서여초에서도이제무작정미러링하고남...,0
2,2022-03-09,Manyong_123,1,1번남들도악착같이살아보자2번남들은니들업보야새끼들아,0
3,2022-03-09,Ops_the_Ops,0,1번남명상같은걸당직자가올린다거나하던분들이혐오로당선됐다하니까어지러움,0
4,2022-03-09,play_with_bow,1,1번남이하루아침에완성되는줄아나윤석열됐다며여가부폐지진짜하려나하고물으면눈반짝이면서아쌉가...,1
...,...,...,...,...,...
6393,2022-02-26,DDZZ_S2,0,엄마는1번남,0
6394,2022-02-16,pumo311,2,2번여캐지일러스트포지션상주인공이가운데에있고주인공기준오른쪽에서있는거같은데오른팔포지션일...,0
6395,2022-02-13,leemaryellow,113,아로하의선택1번남2번남3번남아스트로아로하,1
6396,2022-02-10,kiimsann,1,오프하면1번남할래,0


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람")
            elif np.argmax(logits) == 2:
                test_eval.append("분노")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 4:
                test_eval.append("중립")
            elif np.argmax(logits) == 5:
                test_eval.append("행복")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오")

        # print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
        return predict_sentence, test_eval[0]



In [ ]:
# #질문 무한반복하기! 0 입력시 종료
# end = 1
# while end == 1 :
#     sentence = input("하고싶은 말을 입력해주세요 : ")
#     if sentence == 0 :
#         break
#     predict(sentence)
#     print("\n")

In [ ]:
answer = []
for i in sentences:
  answer.append(predict(i))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
answer

[('2번남이든1번남이든2번남이라고생각드면걍탈빠하세요개지랄하지말고', '혐오'),
 ('그래도우리긍정적으로생각하자고막판에1번남2번남이슈가되면서여초에서도이제무작정미러링하고남자패는거그만두고1번남이랑연대해야한다는얘기가나왔거든나는이게진짜진짜중요한의제라고생각해',
  '슬픔'),
 ('1번남들도악착같이살아보자2번남들은니들업보야새끼들아', '분노'),
 ('1번남명상같은걸당직자가올린다거나하던분들이혐오로당선됐다하니까어지러움', '슬픔'),
 ('1번남이하루아침에완성되는줄아나윤석열됐다며여가부폐지진짜하려나하고물으면눈반짝이면서아쌉가능이지할거너무뻔함', '혐오'),
 ('1번남들도안전하지못할지도요어쩌면아닌척하고이용할생각을해볼수도있것죠늘하던데로', '공포'),
 ('_정말고생많았구요주위에가까이1번남와같은생각같은편이있다는것을늘생각하구요용기내어줘서고맙습니다', '행복'),
 ('참고로정서방은시스젠더게이남으로17퍼고김모모는집안대대로1번남공식설정', '중립'),
 ('이민가실분들1번남1명씩꼭옆구리에끼고가시기약속', '중립'),
 ('우울하다1번남인지2번남인지모를사실눈에보임남직원둘이랑같이일해야한다는게그중하난선거개표방송개웃겼다는데이런얘기나하고앉아있음', '슬픔'),
 ('그리고이건그냥말하는건데나는정치를장난이라생각하지않거든근데왜자꾸1번남2번남이라고부르는거야이거무슨당신의최애대통령을뽑아라프로듀스인것도아니고그사람들도이름이있잖아너희도이름이있고흔한사람1이아니라이름으로불리는게더좋지않아',
  '혐오'),
 ('1번남지지자아닌데2번남제발하는마음이였구결과에빡쳐서잠설치고출근함', '혐오'),
 ('특히요즘대남성겨우정신차렸는데흐름을탄거든그냥1번남소리가듣고싶었든결국1번을찍긴한건데근데결과론적으로는바뀐게없잖아', '공포'),
 ('선동의결과라서오래못가리라예상합니다곧다시1번남들의추악함은만방에위세를떨칠것', '분노'),
 ('시발1번남이네', '중립'),
 ('웃긴게1번남이든2번남이든한남인디요둘다시러요뭔소리', '혐오'),
 ('아이씨2번남들1번남됐네헷갈리게하여간애잘빠져나가기는흥', '혐오'),
 ('1번남', '공포'

In [ ]:
# 확인용 - 안해도 됨.
for s, t in answer:
  print(s, t)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2번남이모두일베일거란생각을하는1번녀들은스타벅스불매존나열심히했을듯일베가2번남일순있어도2번님이모두일베는아닙니다2번녀도철구와사상을공유하는한남사랑녀도아니고오히려1번남뺏어살생각하지말라는여시1번녀들한남사랑이대단할듯둘다좆같은데 혐오
투표하는날1번남2번남난몇번남이게 중립
진정한1번남이네 중립
1번남박지현을지켜달라요청에박지현등장가보자고이재명출처 중립
1번남어필같아보이지왜꽃남같이걸어간다이녀석 행복
1번남이재명내가만든1나를위해1 중립
어제어디가있었는데군인두명은개표소뭐차리느라와있었단말야근데둘다존나잘생겻더라친구한테조용하게우와1번남이다이랬는데앞에2번남있었음 중립
어젯밤첫대선투표라잠이안온다는사촌동생의카톡넘귀야워1번남2번남때문에정치관심갖게됐다고 슬픔
1번남근데이제투표권이없는 슬픔
그러고보니까강동원이가원조1번남이네 놀람
음1번남인거티내넼 놀람
민주당이저정도로호응에답하는모습을보인적이없고여성의원들앞세워준적도없고1번남들이나도한남이었나조오오오오오금반성한적도없음그발전에여성들이기여해서이뤄낸건데빡머갈취급이부랄 혐오
우리동네자칭보수텃밭인디아빠가1번남2번남드립개좋아하심띠발 행복
우리애들1번남이구나 중립
1번남동진어1번남동진 공포
__언니내노예중하나가1번남이래개웃겨 행복
진심으로드리는말씀인데민주당1번남분들이따위홍보좀그만하세요여자가1번찍으면주어지는보상입니까다른게아니라이런게전형적인2번남마인드고여자들이지금까지모든남자를한남이라고불러왔던이유에요1번뽑았다고님들의여성혐오에면죄부가주어지는게아닙니다 놀람
쿠쿠치여자좋아해서1번남이라염색햇어요 행복
히스레저는1번남인데2번남들이따라함이게맞다 놀람
아니잠깐그럼오즈1번남들임이러지말라고햇잖아요 혐오
_1번남2번남도그렇고무슨음식도빨간거다파랗에보정해오는데야가오가있지걍먹지를말든가 공포
엄마랑투표다녀왓는데엄마랑같이냄조들보면서쟤는1번남쟤는2번남평하고다님엄마아주딸한테못된것만배운 혐오
투표햇떠염1번남서창빈과함께 중립
1번남이도겸 중립
_22번남이어도얼굴이1번남이라태생적으로1번남일수밖에없어짐 공포
딸있으면무조건1번1번남 중립
1번남기델

In [ ]:
# Dataframe 만들기
muti_pred_df = pd.DataFrame(answer, columns=['Text', 'Multi pred'])
muti_pred_df

,Text,Multi pred
0,2번남이든1번남이든2번남이라고생각드면걍탈빠하세요개지랄하지말고,혐오
1,그래도우리긍정적으로생각하자고막판에1번남2번남이슈가되면서여초에서도이제무작정미러링하고남...,슬픔
2,1번남들도악착같이살아보자2번남들은니들업보야새끼들아,분노
3,1번남명상같은걸당직자가올린다거나하던분들이혐오로당선됐다하니까어지러움,슬픔
4,1번남이하루아침에완성되는줄아나윤석열됐다며여가부폐지진짜하려나하고물으면눈반짝이면서아쌉가...,혐오
...,...,...
6393,엄마는1번남,중립
6394,2번여캐지일러스트포지션상주인공이가운데에있고주인공기준오른쪽에서있는거같은데오른팔포지션일...,공포
6395,아로하의선택1번남2번남3번남아스트로아로하,공포
6396,오프하면1번남할래,중립


In [ ]:
# pred_df에다가 멀티 라벨 삽입
pred_df.insert(5, 'Multi pred', muti_pred_df['Multi pred']) # 앞에 숫자가 순서
pred_df

,Datetime,Username,Like Count,Text,pred,Multi pred
0,2022-03-09,mingyulover_k,1,2번남이든1번남이든2번남이라고생각드면걍탈빠하세요개지랄하지말고,0,혐오
1,2022-03-09,kero_commu,1,그래도우리긍정적으로생각하자고막판에1번남2번남이슈가되면서여초에서도이제무작정미러링하고남...,0,슬픔
2,2022-03-09,Manyong_123,1,1번남들도악착같이살아보자2번남들은니들업보야새끼들아,0,분노
3,2022-03-09,Ops_the_Ops,0,1번남명상같은걸당직자가올린다거나하던분들이혐오로당선됐다하니까어지러움,0,슬픔
4,2022-03-09,play_with_bow,1,1번남이하루아침에완성되는줄아나윤석열됐다며여가부폐지진짜하려나하고물으면눈반짝이면서아쌉가...,1,혐오
...,...,...,...,...,...,...
6393,2022-02-26,DDZZ_S2,0,엄마는1번남,0,중립
6394,2022-02-16,pumo311,2,2번여캐지일러스트포지션상주인공이가운데에있고주인공기준오른쪽에서있는거같은데오른팔포지션일...,0,공포
6395,2022-02-13,leemaryellow,113,아로하의선택1번남2번남3번남아스트로아로하,1,공포
6396,2022-02-10,kiimsann,1,오프하면1번남할래,0,중립


In [ ]:
# 날짜  삽입
# muti_pred_df.insert(0, 'Datetime', pred_df['Datetime']) # 앞에 숫자가 순서

In [ ]:
pred_df.to_csv("./1번남_multi.csv", encoding='utf-8-sig',sep=',', index=False)